# **MÓDULO 20 - Projeto de Credit Score - Naive Bayes**


No módulo 17 vocês realizaram a primeira etapa do projeto de crédito de vocês.
Então fizeram o tratamendo dos dados, balancearam as classes, transformaram as variáveis categóricas e separam base de treino e teste.
Nessa aula aplicaremos o algoritmo de naive bayes a base de vocês afim de tentarmos trazer previsões do score de crédito.

**IMPORTANTE:** Não se esqueçam de ao enviar o código de vocês para os tutores, enviarem as bases, pois como cada um de vocês realizou as alterações de tratamento indidualmente o tutor precisa ter acesso aos seus dados individuais.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.naive_bayes import GaussianNB

Durante a aula nossa variável a ser prevista (churn) continha apenas 2 categorias, a base de vocês contém mais. O Naive Bayes pode ser aplicado para problemas de classificação com múltiplas classes da mesma forma que para problemas de classificação binária. O Naive Bayes é um algoritmo de classificação probabilístico que calcula a probabilidade de uma amostra pertencer a cada classe e seleciona a classe com a maior probabilidade como a previsão final.
Em resumo, o Naive Bayes pode ser aplicado da mesma maneira para problemas de classificação com múltiplas classes, e os mesmos princípios se aplicam em termos de treinamento, avaliação e aplicação do modelo.

# 1) Comece carregando as bases de treino (X e y) e teste (X e y).
Verifique se o número de linhas condiz, se as variáveis estão corretas sendo apenas a de score para y e as demais nas bases de X e por último, se Y está balanceada no teste.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X_train_file_path =  '/content/drive/MyDrive/CURSOS/EBAC/Cientista de Dados/20 - Aprendizagem Baysiana/Tarefa/X_train_balanced.csv'
y_train_file_path = '/content/drive/MyDrive/CURSOS/EBAC/Cientista de Dados/20 - Aprendizagem Baysiana/Tarefa/y_train_balanced.csv'
X_test_file_path = '/content/drive/MyDrive/CURSOS/EBAC/Cientista de Dados/20 - Aprendizagem Baysiana/Tarefa/X_test.csv'
y_test_file_path = '/content/drive/MyDrive/CURSOS/EBAC/Cientista de Dados/20 - Aprendizagem Baysiana/Tarefa/y_test.csv'

In [5]:
X_train = pd.read_csv(X_train_file_path, delimiter=',')
y_train = pd.read_csv(y_train_file_path, delimiter=',')
X_test = pd.read_csv(X_test_file_path, delimiter=',')
y_test = pd.read_csv(y_test_file_path, delimiter=',')

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            252 non-null    int64  
 1   Income                         252 non-null    float64
 2   Number_of_Children             252 non-null    int64  
 3   Gender_encoded                 252 non-null    int64  
 4   Status_encoded                 252 non-null    int64  
 5   Home_encoded                   252 non-null    int64  
 6   Education_BACHELOR'S DEGREE    252 non-null    int64  
 7   Education_DOCTORATE            252 non-null    int64  
 8   Education_HIGH SCHOOL DIPLOMA  252 non-null    int64  
 9   Education_MASTER'S DEGREE      252 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 19.8 KB


- X_train balanceada com 10 colunas e 252 linhas. Todas variáveis, menos Score_peso.

In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Score_peso  252 non-null    int64
dtypes: int64(1)
memory usage: 2.1 KB


- y_train balanceada com 1 coluna e 252 linhas. Apenas a variável Score_peso.

In [8]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            41 non-null     int64  
 1   Income                         41 non-null     float64
 2   Number_of_Children             41 non-null     int64  
 3   Gender_encoded                 41 non-null     int64  
 4   Status_encoded                 41 non-null     int64  
 5   Home_encoded                   41 non-null     int64  
 6   Education_BACHELOR'S DEGREE    41 non-null     int64  
 7   Education_DOCTORATE            41 non-null     int64  
 8   Education_HIGH SCHOOL DIPLOMA  41 non-null     int64  
 9   Education_MASTER'S DEGREE      41 non-null     int64  
dtypes: float64(1), int64(9)
memory usage: 3.3 KB


- X_test com 10 colunas e 41 linhas. Todas variáveis, menos Score_peso.

In [9]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Score_peso  41 non-null     int64
dtypes: int64(1)
memory usage: 460.0 bytes


- y_test com 1 coluna e 41 linhas. Apenas a variável Score_peso.

In [10]:
# Contagem de exemplos em y_train (base balanceada)
# LOW = 0, AVERAGE = 1, HIGH = 2
y_train_balanceado = y_train.value_counts()
print("Balanceamento em y_train:")
print(y_train_balanceado)

Balanceamento em y_train:
Score_peso
0             84
1             84
2             84
Name: count, dtype: int64


- Base y_train balanceada, com 84 registros para cada categoria, totalizando as 252 linhas.

In [11]:
# Contagem de exmplos em y_test
# LOW = 0, AVERAGE = 1, HIGH = 2
y_test_balanceado = y_test.value_counts()
print("Balanceamento em y_test:")
print(y_test_balanceado)

Balanceamento em y_test:
Score_peso
2             29
0              6
1              6
Name: count, dtype: int64


- Base y_test original, totalizando 41 linhas.

# 2) Aplique o algoritmo de Naive Bayes aos dados de treinamento.

In [12]:
# Criação de uma instância do classificador Naive Bayes Gaussiano
naive_score = GaussianNB()

# Treinamento do classificador usando os dados de treinamento
naive_score.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

# 3) Faça a avaliação do modelo com os dados de treinamento.
Traga a acurácia, recall e plote a matriz de confusão. Não se esqueça de avaliar com suas palavras o desempenho do modelo, interpretando as métricas.

Dica: Para calcularmos o recall em classificação multi classe precisamos usar o atributo macro:
recall = recall_score(y_train, y_pred_train, average='macro')

In [13]:
# Fazer previsões para os dados de treino
y_pred_train = naive_score.predict(X_train)

In [14]:
# Calcular a Acurácia
accuracy = accuracy_score(y_train, y_pred_train)
print("Acurácia:", accuracy)

Acurácia: 0.9841269841269841


- Acurária de 0.9841269841269841 indica que o modelo acertou 98,41% do total de previsões realizadas, o que é muito bom.

In [15]:
# Calcular o Recall
recall = recall_score(y_train, y_pred_train, average='macro')
print("Recall:", recall)

Recall: 0.9841269841269842


- Recall de 0.9841269841269842 indica que o modelo acertou 98,41% (média) dos exemplos positivos, o que também é muito bom. Como as bases de treino estão balanceadas, é normal os valores encontrados para Acurácia e Recall serem próximos ou até mesmo iguais.

In [16]:
#Matriz de Confusão
conf_matrix = confusion_matrix(y_train, y_pred_train)

# Definindo nomes de classes
class_names = ['0 - LOW', '1 - AVERAGE', '2 - HIGH']

# Plotando a matriz de confusão com Plotly
fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=class_names,
    y=class_names,
    colorscale='Blues',
    showscale=True
)
fig.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predicted labels',
    yaxis_title='True labels',
    font=dict(
        family='Arial',
        size=12,
        color='black'
    )
)

In [17]:
conf_matrix = confusion_matrix(y_train, y_pred_train)

conf_matrix

array([[82,  2,  0],
       [ 1, 83,  0],
       [ 0,  1, 83]])

- Para a categoria **0-LOW**, o modelo obteve **82 acertos** e **2 erros** de previsão (que considerou como sendo da categoria 1-AVERAGE).
- Para a categoria **1-AVERAGE**, o modelo obteve **83 acertos** e **1 erro** de previsão (que considerou como sendo da categoria 0-LOW).
- Para a categoria **2-HIGH**, o modelo obteve **83 acertos** e **1 erro** de previsão (que considerou como sendo da categoria 1-AVERAGE).

In [18]:
248/252

0.9841269841269841

# 4) Aplique o modelo aos dados de teste e realize a avaliação dos resultados, da mesma forma que fez acima. Não se esqueça de avaliar com as suas palavras e comparar o desempenho da base treino com a teste.

In [19]:
# Fazer previsões para os dados de teste
y_pred_test = naive_score.predict(X_test)

In [20]:
# Calcular a Acurácia
accuracy = accuracy_score(y_test, y_pred_test)
print("Acurácia:", accuracy)

Acurácia: 0.975609756097561


- Acurácia treino: 0.9841269841269841
- Acurácia teste: 0.975609756097561
- Assim como no modelo de treino, o modelo de teste apresenta um ótimo desempenho ao analisar os novos dados. Não está ocorrendo overfitting, já que o modelo realmente aprendeu os padrões e conseguiu classificar corretamente os dados novos.

In [21]:
# Calcular o Recall
recall = recall_score(y_test, y_pred_test, average='macro')
print("Recall:", recall)

Recall: 0.9444444444444445


- Recall Treino: 0.9841269841269842
- Recall Teste: 0.9444444444444445
- O modelo teste tem um valor ligeiramente inferior, mas ainda assim muito próximo do valor de treino, o que indica que o modelo está bem equilibrado e consegue identificar corretamente os casos verdadeiros.

- Como a base teste apresenta um número menor de dados, um único erro de previsão pode representar um percentual maior que afeta diretamente a Acurária e Recall do modelo.

In [22]:
# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Definindo nomes de classes
class_names = ['0 - LOW', '1 - AVERAGE', '2 - HIGH']

# Plotando a matriz de confusão com Plotly
fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=class_names,
    y=class_names,
    colorscale='Blues',
    showscale=True
)
fig.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predicted labels',
    yaxis_title='True labels',
    font=dict(
        family='Arial',
        size=12,
        color='black'
    )
)

In [24]:
conf_matrix = confusion_matrix(y_test, y_pred_test)

conf_matrix

array([[ 5,  1,  0],
       [ 0,  6,  0],
       [ 0,  0, 29]])

- Para a categoria **0-LOW**, o modelo obteve **5 acertos** e **1 erro** de previsão (que considerou como sendo da categoria 1-AVERAGE).
- Para a categoria **1-AVERAGE**, o modelo obteve **6 acertos** e **nenhum erro** de previsão.
- Para a categoria **2-HIGH**, o modelo obteve **29 acertos** e **nenhum erro** de previsão.

In [23]:
40/41

0.975609756097561

# 5) Descreva com suas palavras o projeto desenvolvido nessa atividade e qual o nosso objetivo principal ao aplicarmos o algoritmo de naive bayes a base de crédito.
Utilize pelo menos 4 linhas.

Dica: Caso você ainda esteja tendo dificuldade em visualizar a aplicação dos projetos e objetivo, consulte seus tutores!

- O projeto tem como objetivo desenvolver um modelo que auxilie a prever a probabilidade de um cliente ser ou não um bom pagador. Para isso, foi aplicado o algoritmo de Naive Bayes à base de crédito, treinando o modelo para classificar de forma eficaz os clientes em diferentes categorias de score de crédito. Este projeto é essencial para instituições financeiras, pois auxilia nas rotinas de análise, solicitações e concessões de crédito ou empréstimos, contribuindo para uma melhor gestão de riscos.